<a href="https://colab.research.google.com/github/kunalr33/SOC_RlForAgents/blob/main/SOC_WEEK2_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np

In [39]:
# Environment setup
grid_size = 4
grid = [
    [0,  1,  0,  10],
    [0, -1,  0,   2],
    [0,  0,  1,   0],
    [-1,  0,  0,   5]
]
'''Action effects defined as position changes
'U': Up -> (i-1, j)
'D': Down -> (i+1, j)
'L': Left -> (i, j-1)
'R': Right -> (i, j+1)'''
actions = ['U', 'D', 'L', 'R']
action_effects = {'U': (-1, 0), 'D': (1, 0), 'L': (0, -1), 'R': (0, 1)}
gamma = 0.95  # Discount factor
theta = 0.0001  # Convergence threshold

In [40]:
# Initialize value function and policy
V = np.zeros((grid_size, grid_size)) #Vk for random policy
policy = np.random.choice(actions, (grid_size, grid_size))

In [41]:
# Policy Evaluation
def policy_evaluation(policy, V): # current value fn and policy as arg
    while True:
        delta = 0                 # delta track max change in V
        for i in range(grid_size):    # Grid traversal
            for j in range(grid_size):
                v = V[i, j]
                if grid[i][j] == -1:  # Skip obstacle cells
                    continue
                new_value = 0  # Initialize new value for (i,j) cell
                action = policy[i, j]  # Current action for (i,j) cell by current policy
                di, dj = action_effects[action] # change in co-ordinate
                ni, nj = i + di, j + dj         #Next state
                if 0 <= ni < grid_size and 0 <= nj < grid_size: # check if nextState within grid
                    reward = grid[ni][nj]                       #R(t+1)
                    new_value = reward + gamma * V[ni, nj]      #Bellman equation for update
                V[i, j] = new_value
                delta = max(delta, abs(v - V[i, j]))            #To check convergence
        if delta < theta:
            break
    return V

In [42]:
# Policy Improvement
def policy_improvement(V):
    policy_stable = True          # to check if policy remain same or not during improvemnet
    for i in range(grid_size):    #grid traversal
        for j in range(grid_size):
            if grid[i][j] == -1:  # Skip obstacle cells
                continue
            old_action = policy[i, j] # current action according to V, stored to check if action changes
            action_values = []        # empty list to store q
            for action in actions:    # iteration for all possible action
                di, dj = action_effects[action]
                ni, nj = i + di, j + dj
                if 0 <= ni < grid_size and 0 <= nj < grid_size:
                    reward = grid[ni][nj]
                    action_values.append((reward + gamma * V[ni, nj], action))
            best_action = max(action_values, key=lambda x: x[0])[1] #determine best action
            policy[i, j] = best_action                              # update policy
            if old_action != best_action:                           # check if policy changes
                policy_stable = False
    return policy, policy_stable

In [43]:
# Policy Iteration
def policy_iteration():
    global policy
    global V
    while True:
        V = policy_evaluation(policy, V)                  #Returns updated valueFunction
        policy, policy_stable = policy_improvement(V)     #Returns improved policy
        if policy_stable:                                 #Check if policy optimal
            break
    return policy, V                                      #Returns optimal policy and valueFunction

In [44]:
# Display the grid world
def display_policy(policy):
    for i in range(grid_size):
        print(" ".join(policy[i]))

In [45]:
# Visualize the agent's path
def visualize_path(policy, start):  #start: starting position in grid
    path = []                       #sequence of positions the agent follow for optimalPolicy
    i, j = start
    while True:
        path.append((i, j))
        if grid[i][j] in {10, 5, 2, 1}:  # Stop if agent reaches a reward pot
            break
        action = policy[i, j]                  #action and change in position
        di, dj = action_effects[action]
        i, j = i + di, j + dj
        if not (0 <= i < grid_size and 0 <= j < grid_size):  # Stop if agent goes out of grid
            break
    return path

In [46]:
# Run Policy Iteration
optimal_policy, optimal_value_function = policy_iteration()

In [47]:
#Results
print("Optimal Policy:")
display_policy(optimal_policy)
print("Optimal Value Function:")
print(optimal_value_function)

Optimal Policy:
R R R D
U L R U
R R R U
U R R U
Optimal Value Function:
[[111.15059895 115.948069   122.05066555 117.94813227]
 [105.593069     0.         117.94813227 122.05072566]
 [102.07566555 107.44813227 112.05072566 117.94818938]
 [  0.         105.87572566 111.44818938 112.05077991]]


In [48]:
# Visualisation
start = (2, 0)  # Starting point
path = visualize_path(optimal_policy, start)
print("Path taken by the agent:")
print(path)

Path taken by the agent:
[(2, 0), (2, 1), (2, 2)]
